# Data Wrangling

THe NSF is organized by directorate as follows:
1. Directorate for Biological Sciences
2. Directorate for Computer & Information Science & Engineering
3. Directorate for Education & Human Resources
4. Directorate for Engineering
5. Directorate for Geosciences
6. Directorate for Mathematical & Physical Sciences
7. Directorate for Social, Behavioral & Economic Sciences

It seems natural to think those would be topics we can identify based on award's abstracts.
Each directorate has multiple division.
There is also a number of offices which we will group together 
and treat them at the same level as directorate

First, let's take a look at our short element data.

In [52]:
import pandas as pd
import os
import re
import json
from itertools import chain, combinations
from sklearn.feature_extraction import stop_words
from pandas.io.json import json_normalize

In [2]:
# path to short element json file
short_elements_dir = os.path.join(os.pardir,'data', 'interim', 'short_elements.json')

In [3]:
# decode json file
with open(short_elements_dir, encoding='utf-8') as f:
    d = json.load(f)

In [4]:
# create dataframe based on list of dictionaries
df_short = json_normalize(d, meta=['award_id', 'award_instr'])
# remove nested data (some keys have value that contains list of dict)
df_short = df_short.drop(['Institution', 'Investigator', 'ProgramElement'], axis = 1)

In [5]:
df_short.head()

,amount,award_id,award_instr,eff_date,exp_date,nsf_officer,org_code,org_direct,org_div,title
0,125000,6000001,Standard Grant,04/15/1960,03/31/1960,name not available,5010000,Direct For Computer & Info Scie & Enginr,Division of Computing and Communication Founda...,Chemical Education Material Study (G12226)
1,28000,6100002,Standard Grant,12/15/1961,12/31/1962,name not available,5020000,Direct For Computer & Info Scie & Enginr,Div Of Information & Intelligent Systems,Translation and Publication of the 1961 Issues...
2,40160,6100003,Standard Grant,12/15/1961,01/31/1965,name not available,5090000,Direct For Computer & Info Scie & Enginr,Office of Advanced Cyberinfrastructure (OAC),Advanced Science Seminar in Soil Clay Mineralo...
3,,6100004,Standard Grant,02/15/1962,05/31/1966,name not available,5010200,Direct For Computer & Info Scie & Enginr,Division of Computing and Communication Founda...,Development of Science Teaching Materials For ...
4,1334824,6100005,Standard Grant,02/15/1962,09/30/1968,name not available,5010200,Direct For Computer & Info Scie & Enginr,Division of Computing and Communication Founda...,A Project For the Development of the Education...


When an officer name is not available, label it as Nan. The string 'name not available' has different spacing.

In [6]:
# separate each word 
df_short.nsf_officer = df_short.nsf_officer.str.split()
# recombine in a controlled spacing convention
df_short.nsf_officer = df_short.nsf_officer.str.join(' ')
# replace missing officer's name by Nan
df_short.nsf_officer.replace('name not available',pd.np.nan, inplace = True)

Convert date to datetime object.

In [7]:
# convert date string to datetime object
df_short.eff_date = pd.to_datetime(df_short.eff_date, format='%m/%d/%Y')
df_short.exp_date = pd.to_datetime(df_short.exp_date, format='%m/%d/%Y')

Mark missing amount as Nan

In [8]:
df_short.amount.replace('', pd.np.nan, inplace = True)

Mark missing directorate as Nan

In [9]:
df_short.org_direct.replace('',pd.np.nan, inplace = True)

In [10]:
df_short.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438352 entries, 0 to 438351
Data columns (total 10 columns):
amount         438245 non-null float64
award_id       438352 non-null int64
award_instr    438352 non-null object
eff_date       438352 non-null datetime64[ns]
exp_date       438352 non-null datetime64[ns]
nsf_officer    368565 non-null object
org_code       438352 non-null object
org_direct     438222 non-null object
org_div        438352 non-null object
title          438352 non-null object
dtypes: datetime64[ns](2), float64(1), int64(1), object(6)
memory usage: 33.4+ MB


Directorate have different abbreviations. Office are also listed at the same level as directorate.

In [11]:
df_short.org_direct.value_counts()

Direct For Mathematical & Physical Scien                        90586
Directorate For Engineering                                     70718
Direct For Biological Sciences                                  62424
Directorate For Geosciences                                     61959
Direct For Computer & Info Scie & Enginr                        53079
Direct For Education and Human Resources                        39804
Direct For Social, Behav & Economic Scie                        35298
Office Of The Director                                          23277
Office of Budget, Finance, & Award Management                     318
Office Of Information & Resource Mgmt                             198
Directorate for Engineering                                       100
Directorate for Computer & Information Science & Engineering       97
Directorate for Education & Human Resources                        97
Directorate for Social, Behavioral & Economic Sciences             70
Directorate for Biol

Let's consolidate directorate names.

In [12]:
# make sure all names are lower case for comparison
df_short.org_direct = df_short.org_direct.str.lower()

In [13]:
# keep only words
df_short.org_direct = df_short.org_direct.str.findall('\w+')

In [14]:
# remove stopwords
df_short.loc[df_short.org_direct.notnull(), 'org_direct'] = \
    df_short.loc[df_short.org_direct.notnull(), 'org_direct'].apply( \
    lambda x: [word for word in x if word not in stop_words.ENGLISH_STOP_WORDS])

In [15]:
# recombine text
df_short.org_direct = df_short.org_direct.str.join(' ')

In [16]:
# get all possible directorate name
s_direct_names = df_short.org_direct.value_counts()

In [17]:
# derive word count in each directorate name
direct_names_len = {direct: len(direct.split()) for direct in s_direct_names.index}

### Utility function to identify text abbreviations

In [18]:
def are_letters_common(abbr, full_word):
    """
    returns true if all letters in abbreviation abbr are present in full_word
    """
    # check if all letters are in full_word
    for l in list(abbr):
        
        if l not in full_word:
            
            return False
    
    # true if loop completed (all letters in full)
    return True

In [19]:
def find_abbreviations(valcount_dict):
    """
    make pairs of abbreviated, non-abbreviated names
    """
    # make two lists: abbrevation list and replacement list
    abbreviation = []
    replacement = []
    
    # group dict by value
    for w_cnt in range(min(valcount_dict.values()), max(valcount_dict.values())+1):
        
        # make a list of keys which have the same count
        list4pairs = [kp.split() for kp,vp in valcount_dict.items() if vp == w_cnt]
        
        # make a list of pair combinations
        pairs = list(combinations(list4pairs , 2))
        
        # compare pairs
        for p in pairs:
            abbre_list =[]
            repl_list = []
            
            # compare word by word
            for w in range(w_cnt):
                
                # get abbreviated word and longer word (full)
                if len(p[0][w]) >=  len(p[1][w]):
                    abbre_word = p[1][w]
                    full_word = p[0][w]
                else:
                    abbre_word = p[0][w]
                    full_word = p[1][w]
                
                # do they have the same root?
                # test if all letters in abbre_word are in full_word
                if are_letters_common(abbre_word,full_word):
                    abbre_list.append(abbre_word)
                    repl_list.append(full_word)
                else:
                    # root is different, move on
                    # decrement w to indicate loop ended with a break statement
                    w -= 1
                    break
            # if for loop complete, concatenate word list
            if w == w_cnt-1:
                abbreviation.append(' '.join(abbre_list))
                replacement.append(' '.join(repl_list))
                
    # return two list
    return abbreviation,replacement

In [20]:
# figure out directorate name that matches non abrreviated name
abbreviation, replacement = find_abbreviations(direct_names_len)

In [21]:
for a, r in zip(abbreviation, replacement):
    print('{}  --->  {}'.format(a, r))

direct biological sciences  --->  directorate biological sciences
office information resource mgmt  --->  office information resource management
direct education human resources  --->  directorate education human resources
direct mathematical physical scien  --->  directorate mathematical physical sciences
direct computer info scie enginr  --->  directorate computer information science engineering
direct social behav economic scie  --->  directorate social behavioral economic sciences


In [22]:
# replace each abbreviation by full name
df_short.org_direct.replace(to_replace=abbreviation, value=replacement, inplace=True, method='pad')

In [23]:
# remove row where directorate is missing
df_short.dropna(subset=['org_direct'], inplace=True )

In [24]:
# removes all offices instances
df_short = df_short[ ~df_short.org_direct.str.contains('office', case=False) ]

### We did it! Only 7 unique directorates

In [25]:
df_short.org_direct.value_counts()

directorate mathematical physical sciences              90613
directorate engineering                                 70818
directorate biological sciences                         62490
directorate geosciences                                 62011
directorate computer information science engineering    53176
directorate education human resources                   39901
directorate social behavioral economic sciences         35368
Name: org_direct, dtype: int64

In [34]:
# dataframe to merge with abstract
df_direct = df_short.loc[:, ['award_id', 'org_direct', 'eff_date']]

In [35]:
df_direct.head()

,award_id,org_direct,eff_date
0,6000001,directorate computer information science engin...,1960-04-15
1,6100002,directorate computer information science engin...,1961-12-15
2,6100003,directorate computer information science engin...,1961-12-15
3,6100004,directorate computer information science engin...,1962-02-15
4,6100005,directorate computer information science engin...,1962-02-15


### Load abstracts and merge them with directorate

In [27]:
# path to abstract json file
abstracts_dir = os.path.join(os.pardir,'data', 'interim', 'abstracts.json')   

In [31]:
# read abstracts
df_abstract = pd.read_json(abstracts_dir)
df_abtract.tail()

,abstract,award_id
438347,"This project focuses on telomerase, the enzym...",1764273
438348,Modern massive data appear in increasing volum...,1764280
438349,The broader impact/commercial potential of thi...,1764312
438350,This project will examine historical changes i...,1764418
438351,This award will provide support for the attend...,1764420


### Join abstract and directorate

In [36]:
df_full = pd.merge(df_direct, df_abstract, on = 'award_id')

In [40]:
# some abstract were not available, especially early on (60s and 70s)
df_full.dropna(subset=['abstract'], inplace=True)

In [41]:
df_full.shape

(313987, 4)

In [42]:
df_full.head()

,award_id,org_direct,eff_date,abstract
69132,8313753,directorate engineering,1984-11-01,"Due to their proven technology, reciprocating-..."
69171,8313873,directorate engineering,1985-07-15,The goal of this research is to develop a tech...
69510,8315117,directorate engineering,1984-06-01,This research program will investigate the mec...
70146,8317279,directorate engineering,1984-12-01,This research is directed toward increasing th...
70890,8319762,directorate engineering,1984-11-01,This research involves both theoretical and ex...


In [43]:
topic_dir = os.path.join(os.pardir,'data', 'processed', 'topics.csv')   
df_full.to_csv(topic_dir, index=False)

### TODO: Extras

In [5]:
df_investigator = json_normalize(d, record_path='Investigator', meta=['award_id'])

In [21]:
# replace missing names by Nan
df_investigator.FirstName.replace('',pd.np.nan, inplace = True)
df_investigator.LastName.replace('',pd.np.nan, inplace = True)

In [22]:
df_investigator.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664639 entries, 0 to 664638
Data columns (total 4 columns):
FirstName    662868 non-null object
LastName     664599 non-null object
Role         664639 non-null object
award_id     664639 non-null int32
dtypes: int32(1), object(3)
memory usage: 17.7+ MB


In [23]:
df_investigator.head()

,FirstName,LastName,Role,award_id
0,Bart,Holm,Principal Investigator,6000001
1,Jeremy,Taylor,Principal Investigator,6100002
2,Charles,Rich,Principal Investigator,6100003
3,Jeremy,Taylor,Principal Investigator,6100004
4,Jeremy,Taylor,Principal Investigator,6100005


In [87]:
def get_unq_names(serie_name):
    """
        Returns set of unique names. Takes cares of barrelled names.
        INPUT: Pandas series of string (object)
    """
    # retrieve unique first names
    names = set(serie_name[serie_name.notnull()])

    # split barrelled names using same pattern used for abstracts
    names = [ re.findall(r'(?u)\b[a-zA-Z][a-zA-Z]+\b', n) for n in names]

    # flatten list of list and keep unique names
    return set(chain.from_iterable(names))

In [90]:
# Retrieve set of first and last names
firstnames = get_unq_names(df_investigator.FirstName)
lastnames = get_unq_names(df_investigator.LastName)

In [92]:
len(firstnames), len(lastnames), len(firstnames)+len(lastnames)

(21747, 75328, 97075)

In [93]:
# consolidates set of names
names = firstnames.union(lastnames)
len(names)

92506

In [99]:
# make one long string to write to file
str_names = '\n'.join(names)
# write to file
with open(os.path.join(os.pardir, 'data', 'external', 'pi_names.txt'), 'w') as f:
    f.write(str_names.lower())

In [44]:
# retrieve institution information
df_institution = json_normalize(d, record_path='Institution', meta=['award_id'])

In [45]:
df_institution.head()

,City,Country,Name,State,StreetAddress,award_id
0,BERKELEY,United States,University of California-Berkeley,CA,Sponsored Projects Office,6000001
1,Washington,United States,The Geochemical Society,DC,Carnegie Institution for Science,6100002
2,BLACKSBURG,United States,Virginia Polytechnic Institute and State Unive...,VA,Sponsored Programs 0170,6100003
3,WASHINGTON,United States,American Association For Advancement Science,DC,"1200 NEW YORK AVENUE, N.W.",6100004
4,ARLINGTON,United States,American Anthropological Assoc,VA,"2300 Clarendon Blvd, Suite 1301",6100005


In [47]:
df_institution.Name.value_counts().count()

19780